# Economic Trend Analysis from 2000-2024

### Importing and Reading Files

In [26]:
import pandas as pd

In [27]:
# File paths for the uploaded files 
files = {
    "Federal_Funds_Effective_Rate": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\Federal Funds Effective Rate.csv",
    "GDP": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\GDP.csv",
    "Median_CPI": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\Median CPI.csv",
    "sp500_yahoo_csv": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\sp500 yahoo.csv",
    "unemployment_rate": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\unemployment_rate.csv"
}

In [28]:
# Function to read the files and capitalize column names
def read_and_capitalize(file_path):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.upper()  # Capitalize all column names
    return df


In [29]:
# Reading and capitalizing all datasets
federal_funds_rate = read_and_capitalize(files["Federal_Funds_Effective_Rate"])
gdp = read_and_capitalize(files["GDP"])
median_cpi = read_and_capitalize(files["Median_CPI"])
sp500 = read_and_capitalize(files["sp500_yahoo_csv"])
unemployment_rate = read_and_capitalize(files["unemployment_rate"])

In [30]:
# Check the head of each file to understand structure 
{
    "Federal Funds Effective Rate": federal_funds_rate.head(),
    "GDP:": gdp.head(),
    "Median CPI": median_cpi.head(),
    "S&P500": sp500.head(),
    "Unemployment_Rate": unemployment_rate.head()
    
}

{'Federal Funds Effective Rate':          DATE   DFF
 0  2000-01-01  3.99
 1  2000-01-02  3.99
 2  2000-01-03  5.43
 3  2000-01-04  5.38
 4  2000-01-05  5.41,
 'GDP:':          DATE        GDP
 0  2000-01-01  10002.179
 1  2000-04-01  10247.720
 2  2000-07-01  10318.165
 3  2000-10-01  10435.744
 4  2001-01-01  10470.231,
 'Median CPI':          DATE  MEDCPIM158SFRBCLE
 0  2000-01-01           3.851771
 1  2000-02-01           2.785584
 2  2000-03-01           3.416809
 3  2000-04-01           2.276901
 4  2000-05-01           2.853534,
 'S&P500':         DATE      OPEN      HIGH       LOW    CLOSE  ADJ CLOSE          VOLUME
 0  29-Dec-23  4,782.88  4,788.43  4,751.99  4,769.83   4,769.83  3,126,060,000
 1  28-Dec-23  4,786.44  4,793.30  4,780.98  4,783.35   4,783.35  2,698,860,000
 2  27-Dec-23  4,773.45  4,785.39  4,768.90  4,781.58   4,781.58  2,748,450,000
 3  26-Dec-23  4,758.86  4,784.72  4,758.45  4,774.75   4,774.75  2,513,910,000
 4  22-Dec-23  4,753.92  4,772.94  4,736.77  4,

### Cleaning the Data

In [31]:
# Standardizing column names for consistency and renaming if necessary
sp500.rename(columns={"DATE": "DATE"}, inplace=True)
median_cpi.rename(columns={"MEDCPIM158SFRBCLE": "MEDIAN_CPI"}, inplace=True)

# Standardizing date formats
for df in [federal_funds_rate, gdp, median_cpi, unemployment_rate]:
    df["DATE"] = pd.to_datetime(df["DATE"])

In [36]:
# Remove duplicates across all datasets
for df in [federal_funds_rate, gdp, median_cpi, sp500, unemployment_rate]:
    df.drop_duplicates(inplace=True)

In [37]:
# Handling missing values by forward filling
for df in [federal_funds_rate, gdp, median_cpi, sp500, unemployment_rate]:
    df.fillna(method='ffill', inplace=True)



In [38]:
# Define output paths for saving cleaned files
output_paths = {
    "Federal_Funds_Effective_Rate": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\Federal_Funds_Effective_Rate_cleaned.csv",
    "GDP": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\GDP_cleaned.csv",
    "Median_CPI": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\Median_CPI_cleaned.csv",
    "S&P500_CSV": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\sp500_yahoo_cleaned.csv",
    "Unemployment_Rate": r"C:\Users\AKKem\OneDrive\Desktop\Data Analysis Modules\Projects\Economic_Trend_Analysis\Resources\unemployment_rate_cleaned.csv"
}

# Save cleaned datasets
federal_funds_rate.to_csv(output_paths["Federal_Funds_Effective_Rate"], index=False)
gdp.to_csv(output_paths["GDP"], index=False)
median_cpi.to_csv(output_paths["Median_CPI"], index=False)
sp500.to_csv(output_paths["S&P500_CSV"], index=False)
unemployment_rate.to_csv(output_paths["Unemployment_Rate"], index=False)

print("All cleaned files have been saved successfully to the Resources folder!")

All cleaned files have been saved successfully to the Resources folder!


### Exploratory Data Analysis